Файлы для соревнования
* hotels_train.csv - набор данных для обучения
* hotels_test.csv - набор данных для оценки качества
* submission.csv - файл сабмишна в нужном формате

Признаки
* hotel_address - адрес отеля
* review_date - дата, когда рецензент разместил соответствующий отзыв.
* average_score - средний балл отеля, рассчитанный на основе последнего комментария за последний год
* hotel_name - название отеля
* reviewer_nationality - национальность рецензента
* negative_review - отрицательный отзыв, который рецензент дал отелю.
* review_total_negative_word_counts - общее количество слов в отрицательном отзыв
* positive_review - положительный отзыв, который рецензент дал отелю
* review_total_positive_word_counts - общее количество слов в положительном отзыве
* reviewer_score - оценка, которую рецензент поставил отелю на основе своего опыта
* total_number_of_reviews_reviewer_has_given - количество отзывов, которые рецензенты дали в прошлом
* total_number_of_reviews - общее количество действительных отзывов об отеле
* tags - теги, которые рецензент дал отелю.
* days_since_review - продолжительность между датой проверки и датой очистки
* additional_number_of_scoring - есть также некоторые гости, которые просто поставили оценку сервису, а не оставили отзыв. Это число указывает, сколько там действительных оценок без проверки.
* lat - широта отеля
* lng - долгота отеля

In [83]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
# Загружаем специальный инструмент для разбивки:  
from sklearn.model_selection import train_test_split
# Импортируем необходимые библиотеки:  
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели  
from sklearn import metrics # инструменты для оценки точности модели 
from category_encoders import  OneHotEncoder, OrdinalEncoder
import category_encoders as ce
import re
RANDOM_SEED = 42

In [84]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize

from nltk.corpus import stopwords
nltk.download('stopwords')

from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

from nltk.corpus import wordnet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
from nltk import pos_tag

import string

from nltk.tokenize import WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Home\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [85]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [86]:
DATA_DIR = 'D:\IDE\data\EDA\PROJECT-3. EDA + Feature Engineering. Соревнование на Kaggle/'
df_train = pd.read_csv(DATA_DIR+'hotels_train.csv', sep=',') 
df_test = pd.read_csv(DATA_DIR+'hotels_test.csv', sep=',') 
sample_submission = pd.read_csv(DATA_DIR+'/submission.csv', sep=',')

In [87]:
display(df_test.head())
display(df_test.info())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520
3,Piazza Della Repubblica 17 Central Station 201...,241,10/17/2015,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",656 day,45.479888,9.196298
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,5/16/2016,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,7,2,"[' Business trip ', ' Solo traveler ', ' Class...",444 day,52.370545,4.888644


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 16 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               128935 non-null  object 
 1   additional_number_of_scoring                128935 non-null  int64  
 2   review_date                                 128935 non-null  object 
 3   average_score                               128935 non-null  float64
 4   hotel_name                                  128935 non-null  object 
 5   reviewer_nationality                        128935 non-null  object 
 6   negative_review                             128935 non-null  object 
 7   review_total_negative_word_counts           128935 non-null  int64  
 8   total_number_of_reviews                     128935 non-null  int64  
 9   positive_review                             128935 non-null  object 
 

None

In [88]:
display(df_train.head())
display(df_train.info())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,reviewer_score,tags,days_since_review,lat,lng
0,Stratton Street Mayfair Westminster Borough Lo...,581,2/19/2016,8.4,The May Fair Hotel,United Kingdom,Leaving,3,1994,Staff were amazing,4,7,10.0,"[' Leisure trip ', ' Couple ', ' Studio Suite ...",531 day,51.507894,-0.143671
1,130 134 Southampton Row Camden London WC1B 5AF...,299,1/12/2017,8.3,Mercure London Bloomsbury Hotel,United Kingdom,poor breakfast,3,1361,location,2,14,6.3,"[' Business trip ', ' Couple ', ' Standard Dou...",203 day,51.521009,-0.123097
2,151 bis Rue de Rennes 6th arr 75006 Paris France,32,10/18/2016,8.9,Legend Saint Germain by Elegancia,China,No kettle in room,6,406,No Positive,0,14,7.5,"[' Leisure trip ', ' Solo traveler ', ' Modern...",289 day,48.845377,2.325643
3,216 Avenue Jean Jaures 19th arr 75019 Paris Fr...,34,9/22/2015,7.5,Mercure Paris 19 Philharmonie La Villette,United Kingdom,No Negative,0,607,Friendly staff quiet comfortable room spotles...,11,8,10.0,"[' Leisure trip ', ' Solo traveler ', ' Standa...",681 day,48.888697,2.394540
4,Molenwerf 1 1014 AG Amsterdam Netherlands,914,3/5/2016,8.5,Golden Tulip Amsterdam West,Poland,Torn sheets,4,7586,The staff was very friendly and helpful Break...,20,10,9.6,"[' Business trip ', ' Couple ', ' Standard Dou...",516 day,52.385601,4.847060


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 386803 entries, 0 to 386802
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               386803 non-null  object 
 1   additional_number_of_scoring                386803 non-null  int64  
 2   review_date                                 386803 non-null  object 
 3   average_score                               386803 non-null  float64
 4   hotel_name                                  386803 non-null  object 
 5   reviewer_nationality                        386803 non-null  object 
 6   negative_review                             386803 non-null  object 
 7   review_total_negative_word_counts           386803 non-null  int64  
 8   total_number_of_reviews                     386803 non-null  int64  
 9   positive_review                             386803 non-null  object 
 

None

In [89]:
display(sample_submission.head())
display(sample_submission.info())

,reviewer_score,id
0,1,488440
1,10,274649
2,3,374688
3,10,404352
4,9,451596


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128935 entries, 0 to 128934
Data columns (total 2 columns):
 #   Column          Non-Null Count   Dtype
---  ------          --------------   -----
 0   reviewer_score  128935 non-null  int64
 1   id              128935 non-null  int64
dtypes: int64(2)
memory usage: 2.0 MB


None

In [90]:
# объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем трейн
df_test['sample'] = 0 # помечаем тест
# в тесте у нас нет значения reviewer_score, заполняем нулями:
df_test['reviewer_score'] = 0 
hotels = df_test.append(df_train, sort=False).reset_index(drop=True)

C:\Users\Home\AppData\Local\Temp\ipykernel_12644\3374675647.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hotels = df_test.append(df_train, sort=False).reset_index(drop=True)


In [91]:
display(hotels.head())
display(hotels.info())
hotels.nunique(dropna=False)

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,total_number_of_reviews_reviewer_has_given,tags,days_since_review,lat,lng,sample,reviewer_score
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,1,"[' Leisure trip ', ' Couple ', ' Double Room '...",13 days,45.533137,9.171102,0,0.0
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,0,7,"[' Leisure trip ', ' Group ', ' Triple Room ',...",234 day,52.385649,4.834443,0,0.0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,5,15,"[' Business trip ', ' Solo traveler ', ' Twin ...",616 day,41.393192,2.161520,0,0.0
3,Piazza Della Repubblica 17 Central Station 201...,241,10/17/2015,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,9,"[' Leisure trip ', ' Couple ', ' Ambassador Ju...",656 day,45.479888,9.196298,0,0.0
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,5/16/2016,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,7,2,"[' Business trip ', ' Solo traveler ', ' Class...",444 day,52.370545,4.888644,0,0.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 18 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   hotel_address                               515738 non-null  object 
 1   additional_number_of_scoring                515738 non-null  int64  
 2   review_date                                 515738 non-null  object 
 3   average_score                               515738 non-null  float64
 4   hotel_name                                  515738 non-null  object 
 5   reviewer_nationality                        515738 non-null  object 
 6   negative_review                             515738 non-null  object 
 7   review_total_negative_word_counts           515738 non-null  int64  
 8   total_number_of_reviews                     515738 non-null  int64  
 9   positive_review                             515738 non-null  object 
 

None

hotel_address                                   1493
additional_number_of_scoring                     480
review_date                                      731
average_score                                     34
hotel_name                                      1492
reviewer_nationality                             227
negative_review                               330011
review_total_negative_word_counts                402
total_number_of_reviews                         1142
positive_review                               412601
review_total_positive_word_counts                365
total_number_of_reviews_reviewer_has_given       198
tags                                           55242
days_since_review                                731
lat                                             1473
lng                                             1473
sample                                             2
reviewer_score                                    38
dtype: int64

In [92]:
print(hotels.isnull().sum())

hotel_address                                    0
additional_number_of_scoring                     0
review_date                                      0
average_score                                    0
hotel_name                                       0
reviewer_nationality                             0
negative_review                                  0
review_total_negative_word_counts                0
total_number_of_reviews                          0
positive_review                                  0
review_total_positive_word_counts                0
total_number_of_reviews_reviewer_has_given       0
tags                                             0
days_since_review                                0
lat                                           3268
lng                                           3268
sample                                           0
reviewer_score                                   0
dtype: int64


In [93]:
# заполняем пропуски самым простым способом
hotels = hotels.fillna(0)

In [94]:
print(hotels['reviewer_nationality'].nunique())
print(hotels['reviewer_nationality'].value_counts())

# удалим пробелы в начале и конце строки 'reviewer_nationality'
hotels['reviewer_nationality'] = hotels['reviewer_nationality'].apply(lambda x: x.strip())

bin_encoder = ce.BinaryEncoder(cols=['reviewer_nationality']) # указываем столбец для кодирования
type_bin = bin_encoder.fit_transform(hotels['reviewer_nationality'])
hotels = pd.concat([hotels, type_bin], axis=1)
display(hotels.head())

227
 United Kingdom               245246
 United States of America      35437
 Australia                     21686
 Ireland                       14827
 United Arab Emirates          10235
                               ...  
 Comoros                           1
 Anguilla                          1
 Tuvalu                            1
 Cape Verde                        1
 Palau                             1
Name: reviewer_nationality, Length: 227, dtype: int64


,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,...,sample,reviewer_score,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7
0,Via Senigallia 6 20161 Milan Italy,904,7/21/2017,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,...,0,0.0,0,0,0,0,0,0,0,1
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,12/12/2016,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,...,0,0.0,0,0,0,0,0,0,1,0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,11/26/2015,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,...,0,0.0,0,0,0,0,0,0,1,1
3,Piazza Della Repubblica 17 Central Station 201...,241,10/17/2015,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,...,0,0.0,0,0,0,0,0,1,0,0
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,5/16/2016,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,...,0,0.0,0,0,0,0,0,0,0,1


In [95]:
# создадим признак того, что рецензент является резидентом страны
hotels['country'] = hotels['hotel_address'].apply(lambda x: x.split()[-1])
hotels['country'] = hotels['country'].apply(lambda x: 'United Kingdom' if x == 'Kingdom' else x)

hotels['is_resident'] = hotels.apply(lambda row: 1 if row['reviewer_nationality'] == row['country'] else 0, axis=1)

print(hotels['is_resident'].nunique())

2


In [96]:
# лишнее
nationality_count = len(hotels['reviewer_nationality'].value_counts())
print(nationality_count)

227


In [97]:
hotels['review_date'] = pd.to_datetime(hotels['review_date'], dayfirst=False)

hotels['review_month'] = hotels['review_date'].dt.month

def review_season (month):
    if month in [3, 4, 5]:
        return (2)
    elif month in [6, 7, 8]:
        return (3)
    elif month in [9, 10, 11]:
        return (4)
    elif month in [12, 1, 2]:
        return (1)

hotels['review_season'] = hotels['review_month'].apply(review_season)

display(hotels.head(6))
print(hotels.info())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,reviewer_nationality,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,...,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7,country,is_resident,review_month,review_season
0,Via Senigallia 6 20161 Milan Italy,904,2017-07-21,8.1,Hotel Da Vinci,United Kingdom,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,...,0,0,0,0,0,1,Italy,0,7,3
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,2016-12-12,8.6,Urban Lodge Hotel,Belgium,No tissue paper box was present at the room,10,5018,No Positive,...,0,0,0,0,1,0,Netherlands,0,12,1
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,2015-11-26,8.3,Alexandra Barcelona A DoubleTree by Hilton,Sweden,Pillows,3,351,Nice welcoming and service,...,0,0,0,0,1,1,Spain,0,11,4
3,Piazza Della Repubblica 17 Central Station 201...,241,2015-10-17,9.1,Hotel Principe Di Savoia,United States of America,No Negative,0,1543,Everything including the nice upgrade The Hot...,...,0,0,0,1,0,0,Italy,0,10,4
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,2016-05-16,9.1,Hotel Esther a,United Kingdom,No Negative,0,4687,Lovely hotel v welcoming staff,...,0,0,0,0,0,1,Netherlands,0,5,2
5,Coram Street Camden London WC1N 1HT United Kin...,709,2015-08-13,8.2,Holiday Inn London Bloomsbury,Ecuador,They don t have free wifi,7,2995,The location is perfect if you don t have a l...,...,0,0,0,1,0,1,United Kingdom,0,8,3


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 30 columns):
 #   Column                                      Non-Null Count   Dtype         
---  ------                                      --------------   -----         
 0   hotel_address                               515738 non-null  object        
 1   additional_number_of_scoring                515738 non-null  int64         
 2   review_date                                 515738 non-null  datetime64[ns]
 3   average_score                               515738 non-null  float64       
 4   hotel_name                                  515738 non-null  object        
 5   reviewer_nationality                        515738 non-null  object        
 6   negative_review                             515738 non-null  object        
 7   review_total_negative_word_counts           515738 non-null  int64         
 8   total_number_of_reviews                     515738 non-null  int64        

In [98]:
# УДАЛЕНИЕ ЛИШНИХ СТОЛБЦОВ
hotels = hotels.drop(['reviewer_nationality', 'review_month', 'country'], axis=1)

In [99]:
# лишнее
display(hotels.head())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,...,reviewer_nationality_0,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7,is_resident,review_season
0,Via Senigallia 6 20161 Milan Italy,904,2017-07-21,8.1,Hotel Da Vinci,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,...,0,0,0,0,0,0,0,1,0,3
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,2016-12-12,8.6,Urban Lodge Hotel,No tissue paper box was present at the room,10,5018,No Positive,0,...,0,0,0,0,0,0,1,0,0,1
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,2015-11-26,8.3,Alexandra Barcelona A DoubleTree by Hilton,Pillows,3,351,Nice welcoming and service,5,...,0,0,0,0,0,0,1,1,0,4
3,Piazza Della Repubblica 17 Central Station 201...,241,2015-10-17,9.1,Hotel Principe Di Savoia,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,...,0,0,0,0,0,1,0,0,0,4
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,2016-05-16,9.1,Hotel Esther a,No Negative,0,4687,Lovely hotel v welcoming staff,7,...,0,0,0,0,0,0,0,1,0,2


In [100]:
# признак города из адреса отеля
hotels['city'] = hotels['hotel_address'].apply(lambda x: x.split()[-2])
hotels['city'] = hotels['city'].apply(lambda x: 'London' if x == 'United' else x)

In [101]:
# закодируем признак города
hotels['city'] = hotels['city'].astype('category')
hotels['hotel_city'] = hotels['city'].cat.codes
hotels.drop(['city'], axis = 1, inplace=True)
display(hotels.head(10))

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,...,reviewer_nationality_1,reviewer_nationality_2,reviewer_nationality_3,reviewer_nationality_4,reviewer_nationality_5,reviewer_nationality_6,reviewer_nationality_7,is_resident,review_season,hotel_city
0,Via Senigallia 6 20161 Milan Italy,904,2017-07-21,8.1,Hotel Da Vinci,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,...,0,0,0,0,0,0,1,0,3,3
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,2016-12-12,8.6,Urban Lodge Hotel,No tissue paper box was present at the room,10,5018,No Positive,0,...,0,0,0,0,0,1,0,0,1,0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,2015-11-26,8.3,Alexandra Barcelona A DoubleTree by Hilton,Pillows,3,351,Nice welcoming and service,5,...,0,0,0,0,0,1,1,0,4,1
3,Piazza Della Repubblica 17 Central Station 201...,241,2015-10-17,9.1,Hotel Principe Di Savoia,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,...,0,0,0,0,1,0,0,0,4,3
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,2016-05-16,9.1,Hotel Esther a,No Negative,0,4687,Lovely hotel v welcoming staff,7,...,0,0,0,0,0,0,1,0,2,0
5,Coram Street Camden London WC1N 1HT United Kin...,709,2015-08-13,8.2,Holiday Inn London Bloomsbury,They don t have free wifi,7,2995,The location is perfect if you don t have a l...,26,...,0,0,0,0,1,0,1,0,3,2
6,Empire Way Wembley Brent London HA9 8DS United...,1005,2016-08-18,8.3,Holiday Inn London Wembley,Room generally a bit shabby with some lack of...,35,3469,Location price It did not cost much more to h...,23,...,0,0,0,0,0,0,1,1,3,2
7,1 Shortlands Hammersmith and Fulham London W6 ...,704,2015-08-11,8.3,Novotel London West,Executive rooms 9th Floor don t have a bath T...,52,2443,Comphy bed upgraded to executive room with ne...,42,...,0,0,0,0,1,1,0,0,3,2
8,35 Rue Caumartin 9th arr 75009 Paris France,211,2016-06-25,8.9,Hotel Saint Petersbourg Opera,Pity about the two days of rain,8,2412,Its centrality proximity to our destination,7,...,0,0,0,0,1,1,1,0,3,4
9,49 Gloucester Place Marble Arch Westminster Bo...,61,2015-09-30,7.4,St George Hotel,Didn t like it at all construction was in pro...,18,334,Didn t like anything about the stay if i had ...,25,...,0,0,0,1,0,0,0,0,4,2


In [102]:
object_columns = [s for s in hotels.columns if hotels[s].dtypes == 'object']
print(object_columns)

['hotel_address', 'hotel_name', 'negative_review', 'positive_review', 'tags', 'days_since_review']


In [103]:
# лишнее
hotels['hotel_name'].nunique()

1492

In [104]:
# лишнее
print(list(hotels['hotel_name'].value_counts(ascending=False)[:10].index))

['Britannia International Hotel Canary Wharf', 'Strand Palace Hotel', 'Park Plaza Westminster Bridge London', 'Copthorne Tara Hotel London Kensington', 'DoubleTree by Hilton Hotel London Tower of London', 'Grand Royale London Hyde Park', 'Holiday Inn London Kensington', 'Hilton London Metropole', 'Millennium Gloucester Hotel London', 'Intercontinental London The O2']


In [105]:
bin_encoder2 = ce.BinaryEncoder(cols=['hotel_name']) # указываем столбец для кодирования
type_bin2 = bin_encoder2.fit_transform(hotels['hotel_name'])
hotels = pd.concat([hotels, type_bin2], axis=1)
display(hotels.head())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,...,hotel_name_1,hotel_name_2,hotel_name_3,hotel_name_4,hotel_name_5,hotel_name_6,hotel_name_7,hotel_name_8,hotel_name_9,hotel_name_10
0,Via Senigallia 6 20161 Milan Italy,904,2017-07-21,8.1,Hotel Da Vinci,Would have appreciated a shop in the hotel th...,52,16670,Hotel was great clean friendly staff free bre...,62,...,0,0,0,0,0,0,0,0,0,1
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,2016-12-12,8.6,Urban Lodge Hotel,No tissue paper box was present at the room,10,5018,No Positive,0,...,0,0,0,0,0,0,0,0,1,0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,2015-11-26,8.3,Alexandra Barcelona A DoubleTree by Hilton,Pillows,3,351,Nice welcoming and service,5,...,0,0,0,0,0,0,0,0,1,1
3,Piazza Della Repubblica 17 Central Station 201...,241,2015-10-17,9.1,Hotel Principe Di Savoia,No Negative,0,1543,Everything including the nice upgrade The Hot...,27,...,0,0,0,0,0,0,0,1,0,0
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,2016-05-16,9.1,Hotel Esther a,No Negative,0,4687,Lovely hotel v welcoming staff,7,...,0,0,0,0,0,0,0,1,0,1


In [106]:
# приведем к нижнему регистру и уберем пробелы в начале и конце отзыва
hotels['positive_review'] = hotels['positive_review'].apply(lambda x: x.strip().lower())
hotels['negative_review'] = hotels['negative_review'].apply(lambda x: x.strip().lower())

# удалим 'no negative' или 'no positive' из текста
hotels['positive_review'] = hotels['positive_review'].apply(lambda x: x.replace('no positive', ''))
hotels['negative_review'] = hotels['negative_review'].apply(lambda x: x.replace('no negative', ''))

display(hotels.head())

,hotel_address,additional_number_of_scoring,review_date,average_score,hotel_name,negative_review,review_total_negative_word_counts,total_number_of_reviews,positive_review,review_total_positive_word_counts,...,hotel_name_1,hotel_name_2,hotel_name_3,hotel_name_4,hotel_name_5,hotel_name_6,hotel_name_7,hotel_name_8,hotel_name_9,hotel_name_10
0,Via Senigallia 6 20161 Milan Italy,904,2017-07-21,8.1,Hotel Da Vinci,would have appreciated a shop in the hotel tha...,52,16670,hotel was great clean friendly staff free brea...,62,...,0,0,0,0,0,0,0,0,0,1
1,Arlandaweg 10 Westpoort 1043 EW Amsterdam Neth...,612,2016-12-12,8.6,Urban Lodge Hotel,no tissue paper box was present at the room,10,5018,,0,...,0,0,0,0,0,0,0,0,1,0
2,Mallorca 251 Eixample 08008 Barcelona Spain,46,2015-11-26,8.3,Alexandra Barcelona A DoubleTree by Hilton,pillows,3,351,nice welcoming and service,5,...,0,0,0,0,0,0,0,0,1,1
3,Piazza Della Repubblica 17 Central Station 201...,241,2015-10-17,9.1,Hotel Principe Di Savoia,,0,1543,everything including the nice upgrade the hote...,27,...,0,0,0,0,0,0,0,1,0,0
4,Singel 303 309 Amsterdam City Center 1012 WJ A...,834,2016-05-16,9.1,Hotel Esther a,,0,4687,lovely hotel v welcoming staff,7,...,0,0,0,0,0,0,0,1,0,1


In [107]:
print(hotels['positive_review'].nunique())
print('Positive:\n', list(hotels['positive_review'].value_counts(ascending=False)[:100].index))
print(hotels['negative_review'].nunique())
print('Negative:\n', list(hotels['negative_review'].value_counts(ascending=False)[:100].index))

401606
Positive:
 ['', 'location', 'everything', 'nothing', 'great location', 'the location', 'good location', 'breakfast', 'friendly staff', 'excellent location', 'staff', 'location and staff', 'location staff', 'location location location', 'location was great', 'good breakfast', 'everything was perfect', 'location was good', 'comfy bed', 'perfect location', 'the staff', 'location is great', 'location is good', 'all', 'clean', 'every thing', 'very friendly staff', 'great breakfast', 'breakfast was good', 'excellent breakfast', 'location was excellent', 'comfortable bed', 'very good location', 'cleanliness', 'location breakfast', 'location is perfect', 'helpful staff', 'great location friendly staff', 'location only', 'great staff', 'location is excellent', 'everything was great', 'central location', 'friendly helpful staff', 'staff and location', 'the bed', 'convenient location', 'position', 'location and breakfast', 'location friendly staff', 'good location friendly staff', 'very fr

In [108]:
# лишний признак?

hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: x.strip('days'))
hotels['days_since_review'] = hotels['days_since_review'].apply(lambda x: x.strip())
hotels['days_since_review'] = hotels['days_since_review'].astype('int')
print(hotels['days_since_review'])

0          13
1         234
2         616
3         656
4         444
         ... 
515733    106
515734    171
515735    543
515736     74
515737    363
Name: days_since_review, Length: 515738, dtype: int32


In [109]:
def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def clean_text(text):
    text = [word.strip(string.punctuation) for word in text.split(' ')]
    text = [word for word in text if not any(c.isdigit() for c in word)]
    text = [t for t in text if len(t) > 0]
    text = [t for t in text if len(t) > 1]
    pos_tags = pos_tag(text)
    text = [word_tokenize(text)]
    text = [word for word in text if word not in stopwords.words('english')]
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    text = ' '.join(text)
    return(text)

In [110]:
reviews_df = hotels[["positive_review", "negative_review", "reviewer_score", "sample"]]
display(reviews_df)

,positive_review,negative_review,reviewer_score,sample
0,hotel was great clean friendly staff free brea...,would have appreciated a shop in the hotel tha...,0.0,0
1,,no tissue paper box was present at the room,0.0,0
2,nice welcoming and service,pillows,0.0,0
3,everything including the nice upgrade the hote...,,0.0,0
4,lovely hotel v welcoming staff,,0.0,0
...,...,...,...,...
515733,tr s proche du metro earl s court,,8.8,1
515734,the bed was so comfy i stayed with my boyfrien...,,9.2,1
515735,great location simple check in out nice shower,room is really small but guess is normal in lo...,8.3,1
515736,location and very comfy bed,,9.2,1


In [111]:
reviews_df['review_clean'] = reviews_df['positive_review'].apply(lambda x: clean_text(x))

TypeError: expected string or bytes-like object